In [1]:
import math
import os
import re
from copy import deepcopy
from pathlib import Path

import numpy as np
import scipy.constants
from scipy.constants import physical_constants

import cheetah

In [2]:
os.environ["LCLS_LATTICE"] = str((Path(".").absolute().parent / "lcls-lattice"))
os.environ["LCLS_LATTICE"]

'/Users/jankaiser/Documents/DESY/lcls-lattice'

In [3]:
lattice_file_path = (
    Path("$LCLS_LATTICE") / "bmad" / "models" / "cu_hxr" / "cu_hxr.lat.bmad"
)
lattice_file_path

PosixPath('$LCLS_LATTICE/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [4]:
resolved_lattice_file_path = Path(
    *[
        os.environ[part[1:]] if part.startswith("$") else part
        for part in lattice_file_path.parts
    ]
)
resolved_lattice_file_path

PosixPath('/Users/jankaiser/Documents/DESY/lcls-lattice/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [5]:
lines = cheetah.bmad.read_clean_lines(resolved_lattice_file_path)
lines[:20]

['beginning[beta_a] =  5.91253676811640894e+000',
 'beginning[alpha_a] =  3.55631307633660354e+000',
 'beginning[beta_b] =  5.91253676811640982e+000',
 'beginning[alpha_b] =  3.55631307633660398e+000',
 'beginning[e_tot] = 6e6',
 'parameter[geometry] = open',
 'parameter[particle] = electron',
 'beginning[theta_position] = -35*pi/180',
 'beginning[z_position] = 3050.512000 - 1032.60052',
 'beginning[x_position] = 10.44893',
 'setsp = 0',
 'setcus = 0',
 'setal = 0',
 'setda = 0',
 'setxleap2 = 0',
 'sethxrss = 0',
 'setsxrss = 0',
 'setcbxfel = 0',
 'setpepx = 0',
 'intgsx = 30.0']

In [6]:
merged_lines = cheetah.bmad.merge_delimiter_continued_lines(
    lines, delimiter="&", remove_delimiter=True
)
merged_lines = cheetah.bmad.merge_delimiter_continued_lines(
    merged_lines, delimiter=",", remove_delimiter=False
)
merged_lines = cheetah.bmad.merge_delimiter_continued_lines(
    merged_lines, delimiter="{", remove_delimiter=False
)
len(lines), len(merged_lines)

(14409, 12215)

In [7]:
context = cheetah.bmad.parse_lines(merged_lines)

In [8]:
context["__use__"]

'cu_hxr'

In [9]:
context["cu_hxr"]

['gunl0a', 'l0al0b', 'lcls2cuh']

In [10]:
context["lcls2cuh"]

['lcls2cuc', 'bsyltuh']

In [11]:
context["lcls2cuc"]

['dl1', 'l1', 'bc1', 'l2', 'bc2', 'l3']

In [12]:
context["dl1"]

['dl1_1', 'dl1_2']

In [13]:
context["dl1_1"]

['begdl1_1',
 'emat',
 'de00',
 'de00a',
 'qe01_full',
 'de01a',
 'im02',
 'de01b',
 'vv02',
 'de01c',
 'qe02_full',
 'dh00',
 'lsrhtr',
 'dh06',
 'tcav0_full',
 'de02',
 'qe03_full',
 'de03a',
 'de03b',
 'sc7',
 'de03c',
 'qe04_full',
 'de04',
 'ws01',
 'de05',
 'otr1',
 'de05c',
 'vv03',
 'de06a',
 'rst1',
 'de06b',
 'ws02',
 'de05a',
 'mrk0',
 'de05a',
 'otr2',
 'de06d',
 'bpm10',
 'de06e',
 'ws03',
 'de05',
 'otr3',
 'de07',
 'qm01_full',
 'de08',
 'sc8',
 'de08a',
 'vv04',
 'de08b',
 'qm02_full',
 'de09',
 'dbmark82',
 'enddl1_1']

In [14]:
context["vv02"]

{'element_type': 'marker'}

In [15]:
def validate_understood_properties(understood: list[str], properties: dict) -> None:
    """
    Validate that all properties are understood. This function primarily ensures that
    properties not understood by Cheetah are not ignored silently.
    """
    for property in properties:
        assert property in understood, (
            f"Property {property} with value {properties[property]} for element"
            f" type {properties['element_type']} is currently not understood."
        )

In [16]:
def convert_element(name: str, properties: dict):
    """
    Convert parsed element dictionary from a Bmad lattice file to a Cheetah element.
    """
    if properties["element_type"] == "drift" or properties["element_type"] == "pipe":
        validate_understood_properties(
            ["element_type", "l", "type", "descrip"], properties
        )
        return cheetah.Drift(name=name, length=properties["l"])
    elif (
        properties["element_type"] == "marker"
        or properties["element_type"] == "monitor"
        or properties["element_type"] == "instrument"
    ):
        validate_understood_properties(["element_type", "type"], properties)

        # TODO: Remove the length if by adding markers to Cheeath
        return cheetah.Drift(name=name, length=0.0)
    elif properties["element_type"] == "quadrupole":
        # TODO: Aperture for quadrupoles?
        validate_understood_properties(
            ["element_type", "l", "k1", "type", "aperture"], properties
        )
        return cheetah.Quadrupole(
            name=name, length=properties["l"], k1=properties["k1"]
        )
    else:
        print(
            f"WARNING: Element of type {properties['element_type']} cannot be converted"
            " correctly. Using drift section instead."
        )
        # TODO: Remove the length if by adding markers to Cheeath
        return cheetah.Drift(
            name=name, length=properties["l"] if "l" in properties else 0.0
        )